# Import Libs and Choose Sizes

In [1]:
import pygame
import random
import time
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import math

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# window size
WIDTH = 1060
HEIGHT = 720
FPS = 30 # how fast game is

# colors
WHITE = (255,255,255)
BLACK = (0,0,0)
RED = (255, 0, 0) # RGB
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
YELLOW = (255,255,0)

# Player Class

In [3]:
class Player(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface((50,50))
        self.image.fill(WHITE)
     
        self.rect = self.image.get_rect()
        self.rect.centerx = 50
        self.rect.centery = HEIGHT/2
        
        self.x_speed = 0
        self.y_speed = 0
        
    def update(self,action):
        # Controls
        key_state = pygame.key.get_pressed()
                
        if key_state[pygame.K_w] or action == 0:
            self.y_speed = -15
            
        elif key_state[pygame.K_s] or action == 1:
            self.y_speed = 15
        
        elif key_state[pygame.K_a] or action == 2:
            self.x_speed = -15
            
        elif key_state[pygame.K_d] or action == 3:
            self.x_speed = 15
            
        
        #Control update 
        self.rect.x += self.x_speed
        self.rect.y += self.y_speed
        
        self.x_speed = 0
        self.y_speed = 0
        
        #Screeen options
        if self.rect.right > WIDTH:
            self.rect.right = WIDTH
            
        if self.rect.left < 0:
            self.rect.left = 0
        
        if self.rect.top < 0:
            self.rect.top = 0
        
        if self.rect.bottom > HEIGHT:
            self.rect.bottom = HEIGHT

    def getCordinates(self):
        return (self.rect.x,self.rect.y)

# Enemy Class

In [4]:
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.image = pygame.Surface((40,40))
        self.image.fill(RED)
      
        self.rect = self.image.get_rect()
        self.y_center = random.randint(15,HEIGHT-10)
        self.x_center = random.randint(WIDTH-400,WIDTH-20)
        self.rect.center = (self.x_center,self.y_center)
        
        self.x_speed = -10
        self.y_speed = 0
             
    def update(self):
        self.rect.x += self.x_speed
        
        if self.rect.left < 0:
            self.y_center = random.randint(15,HEIGHT-10)
            self.x_center = random.randint(WIDTH-400,WIDTH-20)
            self.rect.center = (self.x_center,self.y_center)
            
    def getCordinates(self):
        return (self.rect.x,self.rect.y)

# Agent Class

In [5]:
class DQLAgent:
    def __init__(self):
        # parameter / hyperparameter
        self.state_size = 20   # [distances]
        self.action_size = 4   # move left, move right, move top, move bottom
        
        self.gamma = 0.95
        self.learning_rate = 0.001 
        
        self.epsilon = 1  # explore
        self.epsilon_decay = 0.995
        self.epsilon_min = 0.01
        
        self.memory = deque(maxlen = 1000) 
        
        self.model = self.build_model()
        
        
    def build_model(self):
        # neural network for deep q learning
        model = Sequential()
        model.add(Dense(48, input_dim = self.state_size, activation = "relu"))
        model.add(Dense(self.action_size,activation = "linear"))
        model.compile(loss = "mse", optimizer = Adam(lr = self.learning_rate))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        # storage
        self.memory.append((state, action, reward, next_state, done))
    
    def act(self, state):
        state = np.array(state)
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])
    
    def replay(self, batch_size):
        # training
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory,batch_size)
        for state, action, reward, next_state, done in minibatch:
            state = np.array(state)
            next_state = np.array(next_state)
            if done:
                target = reward 
            else:
                target = reward + self.gamma*np.amax(self.model.predict(next_state)[0])
            train_target = self.model.predict(state)
            train_target[0][action] = target 
            self.model.fit(state,train_target, verbose = 0)
            
    def adaptiveEGreedy(self):
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

# Env Class

In [6]:
class Env:
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.player_group = pygame.sprite.Group()
        self.enemy_group = pygame.sprite.Group()
        self.player = Player()
        self.player_group.add(self.player)
        #enemies
        self.enemy_1 = Enemy()
        self.enemy_group.add(self.enemy_1)
        self.enemy_2 = Enemy()
        self.enemy_group.add(self.enemy_2)
        self.enemy_3 = Enemy()
        self.enemy_group.add(self.enemy_3)
        self.enemy_4 = Enemy()
        self.enemy_group.add(self.enemy_4)        
        self.enemy_5 = Enemy()
        self.enemy_group.add(self.enemy_5)
        self.enemy_6 = Enemy()
        self.enemy_group.add(self.enemy_6)
        self.enemy_7 = Enemy()
        self.enemy_group.add(self.enemy_7)
        self.enemy_8 = Enemy()
        self.enemy_group.add(self.enemy_8)
        self.enemy_9 = Enemy()
        self.enemy_group.add(self.enemy_9)
        self.enemy_10 = Enemy()
        self.enemy_group.add(self.enemy_10)
        
        self.reward = 0
        self.total_reward = 0
        self.done = False
        self.agent = DQLAgent()
        
    def findDistance(self,a, b):
        d = a-b
        return d
    
    def step(self,action):
        state_list = []
        
        #update
        self.player.update(action)
        self.enemy_group.update()
        
        # Get Cordinates
        next_player_states = self.player.getCordinates()
        next_enemy_1_states = self.enemy_1.getCordinates()
        next_enemy_2_states = self.enemy_2.getCordinates()
        next_enemy_3_states = self.enemy_3.getCordinates()
        next_enemy_4_states = self.enemy_4.getCordinates()
        next_enemy_5_states = self.enemy_5.getCordinates()
        next_enemy_6_states = self.enemy_6.getCordinates()
        next_enemy_7_states = self.enemy_7.getCordinates()
        next_enemy_8_states = self.enemy_8.getCordinates()
        next_enemy_9_states = self.enemy_9.getCordinates()
        next_enemy_10_states = self.enemy_10.getCordinates()
        
        #Find Distances
        state_list.append(self.findDistance(next_player_states[0],next_enemy_1_states[0])) #e1 x
        state_list.append(self.findDistance(next_player_states[1],next_enemy_1_states[1])) #e1 y
        state_list.append(self.findDistance(next_player_states[0],next_enemy_2_states[0])) #e2 x
        state_list.append(self.findDistance(next_player_states[1],next_enemy_2_states[1])) #e2 y
        state_list.append(self.findDistance(next_player_states[0],next_enemy_3_states[0])) #e3 x
        state_list.append(self.findDistance(next_player_states[1],next_enemy_3_states[1])) #e3 y
        state_list.append(self.findDistance(next_player_states[0],next_enemy_4_states[0])) #e4 x
        state_list.append(self.findDistance(next_player_states[1],next_enemy_4_states[1])) #e4 y
        state_list.append(self.findDistance(next_player_states[0],next_enemy_5_states[0])) #e5
        state_list.append(self.findDistance(next_player_states[1],next_enemy_5_states[1])) #e5
        state_list.append(self.findDistance(next_player_states[0],next_enemy_6_states[0])) #e6
        state_list.append(self.findDistance(next_player_states[1],next_enemy_6_states[1])) #e6
        state_list.append(self.findDistance(next_player_states[0],next_enemy_7_states[0])) #e7
        state_list.append(self.findDistance(next_player_states[1],next_enemy_7_states[1])) #e7
        state_list.append(self.findDistance(next_player_states[0],next_enemy_8_states[0])) #e8
        state_list.append(self.findDistance(next_player_states[1],next_enemy_8_states[1])) #e8
        state_list.append(self.findDistance(next_player_states[0],next_enemy_9_states[0])) #e9
        state_list.append(self.findDistance(next_player_states[1],next_enemy_9_states[1])) #e9
        state_list.append(self.findDistance(next_player_states[0],next_enemy_10_states[0])) #e10
        state_list.append(self.findDistance(next_player_states[1],next_enemy_10_states[1])) #e10
        
        return [state_list] #len(state_list) = 20
    
    def initialStates(self): #reset()
        self.player_group = pygame.sprite.Group()
        self.enemy_group = pygame.sprite.Group()
        self.player = Player()
        self.player_group.add(self.player)
        #enemies
        self.enemy_1 = Enemy()
        self.enemy_group.add(self.enemy_1)
        self.enemy_2 = Enemy()
        self.enemy_group.add(self.enemy_2)
        self.enemy_3 = Enemy()
        self.enemy_group.add(self.enemy_3)
        self.enemy_4 = Enemy()
        self.enemy_group.add(self.enemy_4)        
        self.enemy_5 = Enemy()
        self.enemy_group.add(self.enemy_5)
        self.enemy_6 = Enemy()
        self.enemy_group.add(self.enemy_6)
        self.enemy_7 = Enemy()
        self.enemy_group.add(self.enemy_7)
        self.enemy_8 = Enemy()
        self.enemy_group.add(self.enemy_8)
        self.enemy_9 = Enemy()
        self.enemy_group.add(self.enemy_9)
        self.enemy_10 = Enemy()
        self.enemy_group.add(self.enemy_10)
        
        self.reward = 0
        self.total_reward = 0
        self.done = False
        
        #state
        state_list = []
        # Get Cordinates
        player_states = self.player.getCordinates()
        enemy_1_states = self.enemy_1.getCordinates()
        enemy_2_states = self.enemy_2.getCordinates()
        enemy_3_states = self.enemy_3.getCordinates()
        enemy_4_states = self.enemy_4.getCordinates()
        enemy_5_states = self.enemy_5.getCordinates()
        enemy_6_states = self.enemy_6.getCordinates()
        enemy_7_states = self.enemy_7.getCordinates()
        enemy_8_states = self.enemy_8.getCordinates()
        enemy_9_states = self.enemy_9.getCordinates()
        enemy_10_states = self.enemy_10.getCordinates()
        
        #Find Distances
        state_list.append(self.findDistance(player_states[0],enemy_1_states[0])) #e1
        state_list.append(self.findDistance(player_states[1],enemy_1_states[1])) #e1
        state_list.append(self.findDistance(player_states[0],enemy_2_states[0])) #e2
        state_list.append(self.findDistance(player_states[1],enemy_2_states[1])) #e2
        state_list.append(self.findDistance(player_states[0],enemy_3_states[0])) #e3
        state_list.append(self.findDistance(player_states[1],enemy_3_states[1])) #e3
        state_list.append(self.findDistance(player_states[0],enemy_4_states[0])) #e4
        state_list.append(self.findDistance(player_states[1],enemy_4_states[1])) #e4
        state_list.append(self.findDistance(player_states[0],enemy_5_states[0])) #e5
        state_list.append(self.findDistance(player_states[1],enemy_5_states[1])) #e5
        state_list.append(self.findDistance(player_states[0],enemy_6_states[0])) #e6
        state_list.append(self.findDistance(player_states[1],enemy_6_states[1])) #e6
        state_list.append(self.findDistance(player_states[0],enemy_7_states[0])) #e7
        state_list.append(self.findDistance(player_states[1],enemy_7_states[1])) #e7
        state_list.append(self.findDistance(player_states[0],enemy_8_states[0])) #e8
        state_list.append(self.findDistance(player_states[1],enemy_8_states[1])) #e8
        state_list.append(self.findDistance(player_states[0],enemy_9_states[0])) #e9
        state_list.append(self.findDistance(player_states[1],enemy_9_states[1])) #e9
        state_list.append(self.findDistance(player_states[0],enemy_10_states[0])) #e10
        state_list.append(self.findDistance(player_states[1],enemy_10_states[1])) #e10
        
        return [state_list] #state
    
    def run(self):
        # Game Loop
        state = self.initialStates()
        batch_size = 16
        running = True
        while running:
            self.reward = 1
            # keep loop running at the right speed
            clock.tick(FPS)
            
            # process input
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False

            #update
            action = self.agent.act(state)
            next_state = self.step(action)
            self.total_reward += self.reward

            # check to see if a enemy hit the player
            hits = pygame.sprite.spritecollide(self.player, self.enemy_group, False)
            if hits: #hits == True
                self.reward = -150
                self.total_reward += self.reward
                self.done = True
                running = False
                print("Total Reward: ", self.total_reward)
            
            # remember (storage)
            self.agent.remember(state,action,self.reward, next_state, self.done)
            
            # update state
            state = next_state
            
            # training
            self.agent.replay(batch_size)
            
            # epsilon greedy
            self.agent.adaptiveEGreedy()                       
                
            #draw / render(show)
            screen.fill(BLACK)
            self.player_group.draw(screen)
            self.enemy_group.draw(screen)

            #after drawing flip display
            pygame.display.flip()
                    
        pygame.quit()  

In [ ]:
if __name__ == '__main__':
    env = Env()
    liste = []
    t = 0
    while True:
        t += 1
        print("Episode:",t)
        liste.append(env.total_reward)
        
        # initialize pygame and create window
        pygame.init()
        screen = pygame.display.set_mode((WIDTH,HEIGHT))
        pygame.display.set_caption("kacma oyunu")
        clock = pygame.time.Clock()
        
        env.run()

<ipython-input-3-00f87673f482>:9: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  self.rect.centery = HEIGHT/2


Episode: 1
